Consider first a mixture of 1-d Gaussians with pdf

$$ f(x) = \sum_{k=1}^K \omega_k \, \frac{\lambda_k^{1/2}}{\sqrt{2 \pi}} \exp \left \{ - \frac12 \lambda_k (x - \mu_k)^2 \right \} ~~ \text{where} ~ \sum_{k=1}^K \omega_k = 1 $$
are the mixture weights, $\mu_1, \ldots, \mu_K$ are the mixture means and $\lambda_1, \ldots, \lambda_K$ are the mixture precisions. 


Suppose we have observations $x^1, \ldots, x^N$. 
Then for $r \in \{1, \ldots, K\}$,
\begin{eqnarray}
- \partial_{\mu_r} \log f(x) & = & - \frac{\partial}{\partial \mu_r} \log \left [ \sum_{k=1}^K \omega_k \lambda_k^{1/2} \exp \left \{ - \frac12 \lambda_k (x - \mu_k)^2 \right \} \right ] = \frac{\omega_r \lambda_r^{3/2} \exp \left \{ - \frac12 \lambda_k (x - \mu_k)^2 \right \} (x - \mu_r)} {\sum_{k=1}^K \omega_k \lambda_k^{1/2} \exp \left \{ - \frac12 \lambda_k (x - \mu_k)^2 \right \}} \\
%
\Rightarrow - \partial_{\mu_r} \sum_{i=1}^N \log f(x^i) & = & \sum_{i=1}^N \frac{\omega_r \lambda_r^{3/2} \exp \left \{ - \frac12 \lambda_k (x^i - \mu_k)^2 \right \} (x^i - \mu_r)} {\sum_{k=1}^K \omega_k \lambda_k^{1/2} \exp \left \{ - \frac12 \lambda_k (x^i - \mu_k)^2 \right \}}. 
\end{eqnarray}

Let $U(\mu) = \log \left ( \sum_{i=1}^N f(x^i) \right )$ be the log-likelihood in terms of the parameters $\mu = (\mu_1, \ldots, \mu_r)$.
Also suppose $\lambda_1 = \cdots = \lambda_k = \lambda$. Then 
\begin{eqnarray} 
- \partial_{\mu_r} U(\mu) 
& = & 
\sum_{i=1}^N \frac{\omega_r \lambda^{3/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_r)^2 \right \}} {\sum_{k=1}^K \omega_k \lambda^{1/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_k)^2 \right \}} (x^i - \mu_r) 
= \sum_{i=1}^N a^i (x^i - \mu_r), ~~ \text{where} ~ a^i = \frac{\omega_r \lambda^{3/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_r)^2 \right \}} {\sum_{k=1}^K \omega_k \lambda^{1/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_k)^2 \right \}} \\
%
\Rightarrow \left \{ \theta \, \partial_{\mu_r} U(\mu) \right \}^+ & 
\leq & 
\lambda \left \{ \sum_{i=1}^N \theta \, a^i \, (x^i - \mu_r) \right \}^+ 
\leq 
\lambda  \sum_{i=1}^N \left \{ \theta \, a^i \, (x^i - \mu_r) \right \}^+ 
\leq 
\lambda \sum_{i=1}^N \left \{ \theta \, (x^i - \mu_r) \right \}^+ ,
\end{eqnarray} 

since $$ \left | \frac{\omega_r \lambda^{3/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_r)^2 \right \} } {\sum_{k=1}^K \omega_k \lambda^{1/2} \exp \left \{ - \frac12 \lambda (x^i - \mu_k)^2 \right \}}  \right | \leq \lambda.$$

Let $m_r(\mu(t)) = \left \{ \theta \, \partial_{\mu_r} U(\mu + \theta \, t) \right \}^+$. Then we want $M_i(t)$ such that $m_r(t) \leq M_r(t)$ for all $t \geq 0$. Therefore, choose $M_r(t) = \sum_{i=1}^N \left \{ \theta \, (x^i - \mu_r - \theta \, t) \right \}^+ = \sum_{i=1}^N \left \{ \theta \, ( y^{(i)} - \theta \, t ) \right \}^+$, where $y^i = x^i - \mu_r$ and $y^{(1)} \leq \cdots \leq y^{(N)}$ are the $y^i$'s sorted. Now let's see what this is. 

If $\theta = 1$, 

$$ M_r(t) =
  \begin{cases}
    \sum_{i=1}^N (y^{(i)} - t)  & \quad \text{if } t \leq y^{(1)} \\
    \sum_{i=1}^k (y^{(i)} - t) + \sum_{i=k+1}^N (t - y^{(i)}) & \quad \text{if } y^{(k)} \leq t \leq y^{(k+1)}, ~~ k = 1, \ldots, N-1 \\
    \sum_{i=1}^N (t - y^{(i)})  & \quad \text{if } t > y^{(N)}
  \end{cases}
$$

Then $\int_0^t M_i(s) \, ds$ is a piecewise quadratic function. 

In [8]:
using Distributions
using StatsBase
using Plots
plotly()

Plots.PlotlyBackend()

In [87]:
function simulate_data(μ, λ, ω, Nobs) 
    ω = ω./sum(ω)
    K = size(μ)[1]
    X = zeros(Nobs) 
    for i in 1:Nobs 
        u = rand(1)[1]
        index = 1
        w = ω[index]
        while u >= w 
            index += 1
            w += ω[index]
        end
        X[i] = rand(Normal(μ[index], 1/sqrt(λ[index]) ))    
    end
    return X
end 

simulate_data (generic function with 1 method)

In [133]:
N = 100
K = 10

ω = rand(K)
λ = ones(K)
μ = 100*(1:10)

X = simulate_data(μ, λ, ω, N);

In [136]:
function derivative_moG(X, μ, ω, λ, r) 
    K = length(μ)
    Nobs = length(X)
    upper = ω[r]*λ[r]^1.5*exp.(-0.5*λ[r]*(X-μ[r]).^2).*(X-μ[r])
    lower = reshape(sum(Diagonal(ω.*λ)*exp.(-0.5*Diagonal(λ)*(transpose(repmat(X,1,K)).-μ).^2),1),Nobs,) 
    return sum(upper ./ lower)
end

derivative_moG (generic function with 1 method)

In [137]:
derivative_moG(X, μ, ω, λ, 4) 

-7.020124769070549

In [113]:
μ_min = -5 
μ_max = 15

15

In [122]:
function computational_bounds(X, μ) 
    return max(sum(X-μ_min)*( sum(X-μ_min)>0), -sum(X-μ_max)*( sum(X-μ_max)<0))
end

computational_bounds (generic function with 1 method)

In [123]:
computational_bounds(X, μ) 

1659.3896941982773

In [138]:
75.00+27.72+535.80+15.77+28.88+437.54+92.39+103.90+21.12

1338.1200000000001